In [45]:
combined_prob = dict()
combined_total_prob = dict()
corrections = dict()

In [46]:
lihkg_csv_file = open("/mnt/vol1/repo/lihkg_corpus/examples/lihkg_6gram-no-suffix.csv", "r")
lihkg_lines = lihkg_csv_file.readlines()

weight = 0.6
for line in lihkg_lines[1:]:
    parsed = line.split(',')
    s = parsed[0]
    p = float(parsed[1])
    total_p = float(parsed[2])
    combined_prob[s] = weight * p
    combined_total_prob[s] = weight * total_p

In [47]:
apple_csv_file = open("/home/opc/apple-daily-corpus/examples/6gram-prob-nosym-lite.csv", "r")
apple_lines = apple_csv_file.readlines()

weight = 1 - weight
for line in apple_lines[1:]:
    parsed = line.split(',')
    s = parsed[0]
    p = float(parsed[1])
    total_p = float(parsed[2])
    combined_prob[s] = combined_prob.get(s, 0) + weight * p
    combined_total_prob[s] = combined_total_prob.get(s, 0) + weight * total_p

In [48]:
correction_file = open("/mnt/vol1/repo/lihkg_corpus/examples/correction.csv", "r")
correction_lines = correction_file.readlines()

parsedHeader = False
for line in correction_lines[1:]:
    line = line.strip()
    if len(line) == 0 or line.startswith("#"): continue;
    if not parsedHeader:
        parsedHeader = True
        continue
    parsed = line.split(',')
    original = parsed[0]
    targets = parsed[1]
    weightMod = 1
    if len(parsed) > 2:
        weightMod = float(parsed[2])
        
    targetTuples = []
    for target in targets.split(" "):
        targetTuples.append([target, weightMod])
        
    targetTuples.append([original, 1 - weightMod])
    
    corrections[original] = targetTuples

In [49]:
result_cond_prob = dict()
result_total_prob = dict()

for ngram_str in combined_prob:
    if ngram_str == '': continue
    if not ngram_str in corrections: 
        cond_prob = combined_prob[ngram_str]
        total_prob = combined_total_prob[ngram_str]
        if not ngram_str in result_cond_prob:
            result_cond_prob[ngram_str] = 0
            result_total_prob[ngram_str] = 0
        result_cond_prob[ngram_str] += cond_prob
        result_total_prob[ngram_str] += total_prob
    else:
        correctionTuples = corrections[ngram_str]
        for correctionTuple in correctionTuples:
            s = correctionTuple[0]
            if not s in result_cond_prob:
                result_cond_prob[s] = 0
                result_total_prob[s] = 0
            cond_prob = combined_prob[ngram_str] * correctionTuple[1]
            total_prob = combined_total_prob[ngram_str] * correctionTuple[1]
            result_cond_prob[s] += cond_prob
            result_total_prob[s] += total_prob

香港 0.9114512117102673


In [50]:
# Dump the result into a csv file.

output_path = "combined.csv"

with open(output_path, "w") as f:
    f.write("ngram,freq,total_freq\n")
    for ngram_str in combined_prob:
        if ngram_str == '': continue
        cond_prob = result_cond_prob[ngram_str]
        total_prob = result_total_prob[ngram_str]
        if cond_prob > 1e-16:
            f.write(ngram_str + "," + str(cond_prob) + "," + str(total_prob) + "\n")